In [3]:
import sys

In [4]:
# we will use motor imaginery data for testing
from pathlib import Path
import importlib

DATA_CLEANER_DIR = Path("../1-data-cleaning-pipelines/motor-imaginary")

def get_motor_imaginery_train_and_validation_data():
    target_dir = DATA_CLEANER_DIR.resolve()
    if str(target_dir) not in sys.path:
        sys.path.append(str(target_dir))

    import centralize_train_data_cleaner 
    importlib.reload(centralize_train_data_cleaner)

    train_labels, train_features, val_labels, val_features = centralize_train_data_cleaner.pipeline()

    return train_labels, train_features, val_labels, val_features

train_labels, train_features, val_labels, val_features = get_motor_imaginery_train_and_validation_data();
train_labels.shape, train_features.shape, val_labels.shape, val_features.shape

[Fetcher] Starting data preparation...
[Download] Skip: BCICIV_2a_gdf.zip already exists
[Extract] Skip: already extracted at /home/kanathipp/Stuffs/Works/final-project-federated-learning/0-raw-data/motor-imaginary/data
[Fetcher] Completed.


(torch.Size([2928]),
 torch.Size([2928, 22, 176]),
 torch.Size([880]),
 torch.Size([880, 22, 176]))

In [18]:
from pathlib import Path
import importlib
MODEL_DIR = Path("../2-models")

def get_eegnet_pytorch_model():
    target_dir = MODEL_DIR.resolve()
    if str(target_dir) not in sys.path:
        sys.path.append(str(target_dir))

    import eegnet_pytorch 
    importlib.reload(eegnet_pytorch)

    return eegnet_pytorch.EEGNet

EEGNet = get_eegnet_pytorch_model()
model = EEGNet(in_channel=64, num_classes=4)

model

EEGNet(
  (block1): Block1(
    (conv_temporal): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
    (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (depthwise): Conv2d(8, 16, kernel_size=(64, 1), stride=(1, 1), groups=8, bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ELU(alpha=1.0)
    (pool): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (drop): Dropout(p=0.5, inplace=False)
  )
  (block2): Block2(
    (sep_depth): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=16, bias=False)
    (sep_point): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ELU(alpha=1.0)
    (pool): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (drop): Dropout(p=0.5, inplace=False)
  )
  (classifier): Classifier(